In [3]:
import sys

!{sys.executable} -m pip install datasets ipywidgets -q

In [4]:
import datasets

In [5]:
ds = datasets.load_dataset('reciprocate/tinygsm_dpo')

Generating test split: 100%|██████████| 309/309 [00:00<00:00, 196429.21 examples/s]


In [18]:
for k in ds:
    print(k, len(ds[k]))

train 5857
test 309


In [10]:
import random
sample = random.choice(ds['train'])

In [16]:
print(list(sample.keys()))

['prompt', 'selected', 'rejected']


In [13]:
print(sample['prompt'])

Consider the following grade-school math problem: Four small boxes can fit in one big box. If 50 big boxes can fit four small boxes each, each having 40 candles, calculate the total number of candles in all small boxes.
Solve this problem using code.
- Give the complete solution to solve the problem written in Python.
- The program should contain multiple lines of code and end with 'result = XXX'.
- Use markdown to format your response starting with '```python' and ending with '```'.


In [14]:
print(sample['rejected'])

Let's solve this problem using Python code.
```python
small_boxes_per_big_box = 4
big_boxes = 50
candles_per_small_box = 40

# Calculate the number of small boxes that can fit in one big box
small_boxes_in_big_box = small_boxes_per_big_box * big_boxes

# Calculate the total number of small boxes
total_small_boxes = big_boxes * small_boxes_in_big_box

# Calculate the total number of candles
total_candles = total_small_boxes * candles_per_small_box
result = total_candles
```



In [17]:
print(sample['selected'])

Let's solve this problem using Python code.
```python
# Define the number of small boxes that can fit in one big box
small_boxes_per_big_box = 4

# Define the number of big boxes that can fit four small boxes each
big_boxes_with_small_boxes = 50

# Define the number of candles in each small box
candles_per_small_box = 40

# Calculate the total number of small boxes
total_small_boxes = big_boxes_with_small_boxes * small_boxes_per_big_box

# Calculate the total number of candles
total_candles = total_small_boxes * candles_per_small_box

result = total_candles
```



In [19]:
def extract_code_from_markdown(s: str) -> list[str]:
    """
    Extract code blocks from markdown formatted text.

    Args:
    - s (str): A string containing markdown formatted text.

    Returns:
    - list (list[str]): A list of code blocks extracted from the markdown.
    """
    code_blocks = []
    in_code_block = False
    current_code_block = []

    for line in s.split('\n'):
        if line.startswith("```python"):
            in_code_block = True
            current_code_block = []
        elif line.startswith("```") and in_code_block:
            in_code_block = False
            code_blocks.append('\n'.join(current_code_block))
            current_code_block = []
        elif in_code_block and not line.startswith('print('):
            current_code_block.append(line)

    return code_blocks

In [24]:
code_blocks = extract_code_from_markdown(sample['selected'])

In [27]:
print(sample['prompt'])

Consider the following grade-school math problem: Four small boxes can fit in one big box. If 50 big boxes can fit four small boxes each, each having 40 candles, calculate the total number of candles in all small boxes.
Solve this problem using code.
- Give the complete solution to solve the problem written in Python.
- The program should contain multiple lines of code and end with 'result = XXX'.
- Use markdown to format your response starting with '```python' and ending with '```'.


In [25]:
print(code_blocks[0])0

# Define the number of small boxes that can fit in one big box
small_boxes_per_big_box = 4

# Define the number of big boxes that can fit four small boxes each
big_boxes_with_small_boxes = 50

# Define the number of candles in each small box
candles_per_small_box = 40

# Calculate the total number of small boxes
total_small_boxes = big_boxes_with_small_boxes * small_boxes_per_big_box

# Calculate the total number of candles
total_candles = total_small_boxes * candles_per_small_box

result = total_candles


## Check TinyGSM8K

In [33]:
!conda install pytorch::pytorch -c pytorch --yes

Channels:
 - pytorch
 - defaults
Platform: linux-64
Solving environment: done

## Package Plan ##

  environment location: /opt/miniconda

  added / updated specs:
    - pytorch::pytorch


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    blas-1.0                   |              mkl           6 KB
    filelock-3.13.1            |  py311h06a4308_0          24 KB
    gmp-6.2.1                  |       h295c915_3         544 KB
    gmpy2-2.1.2                |  py311hc9b5ff0_0         191 KB
    intel-openmp-2023.1.0      |   hdb19cb5_46306        17.2 MB
    jinja2-3.1.3               |  py311h06a4308_0         353 KB
    llvm-openmp-14.0.6         |       h9e868ea_0         4.4 MB
    markupsafe-2.1.3           |  py311h5eee18b_0          25 KB
    mkl-2023.1.0               |   h213fc3f_46344       171.5 MB
    mpc-1.1.0                  |       h10f8cd9_1          90 KB
    mpfr-4.0.2   

In [29]:
!{sys.executable} -m pip install transformers -q

In [30]:
from transformers import pipeline

reward_fn = pipeline(
    "text-classification",
    model="reciprocate/mistral-7b-gsm8k-code-rm",
    truncation=True,
    max_length=4096,
    function_to_apply="none"
)

prompt = """\
Consider the following grade-school math problem: Megan has read 32 books this year. Kelcie has read 1/4 the amount of books that Megan has read. Greg has read 9 more than twice the number of books that Kelcie has read. How many books total have Megan, Kelcie, and Greg read?
Solve this problem using code.
- Give the complete solution to solve the problem written in Python.
- The program should contain multiple lines of code and end with 'result = XXX'.
- Use markdown to format your response starting with '```python' and ending with '```'.
"""

output = """\
Let's solve this problem using Python code.
```python
books_megan = 32
books_kelcie = books_megan / 4
books_kelcie = int(books_kelcie)
books_greg = 2 * books_kelcie + 9
total_books = books_megan + books_kelcie + books_greg
result = total_books```
"""

chats = [[
    {"role": "user", "content": prompt},
    {"role": "assistant", "content":  output}
]]

inputs = [reward_fn.tokenizer.apply_chat_template(chat, tokenize=False) for chat in chats]
output = reward_fn(inputs)
scores = [x["score"] for x in output]

None of PyTorch, TensorFlow >= 2.0, or Flax have been found. Models won't be available and only tokenizers, configuration and file/data utilities can be used.
/home/ubuntu/.local/lib/python3.11/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


RuntimeError: At least one of TensorFlow 2.0 or PyTorch should be installed. To install TensorFlow 2.0, read the instructions at https://www.tensorflow.org/install/ To install PyTorch, read the instructions at https://pytorch.org/.